<a href="https://colab.research.google.com/github/ian-summers/MLB-player-archetypes-by-age/blob/Data-ETL/Manual_ETL_Import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import pyspark
except ImportError:
    !pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ImportCSV").getOrCreate()

In [ ]:

from google.colab import files


uploaded = files.upload()

In [ ]:
import pandas as pd
import io

In [ ]:
csv_files = [
    "batting_2023.csv",
    "batting_2022.csv",
    "batting_2021.csv",
    "batting_2020.csv",
    "batting_2019.csv",
    "batting_2018.csv",
    "batting_2017.csv",
    "batting_2016.csv",
    "batting_2015.csv",
    "batting_2014.csv",
    "batting_2013.csv",
]

# Read each CSV file and create a DataFrame
dataframes = []
for csv_file in csv_files:
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    dataframes.append(df)

# Union all DataFrames to create a single master DataFrame
master_df = dataframes[0]
for df in dataframes[1:]:
    master_df = master_df.union(df)

# Show the master DataFrame
master_df.show()

In [ ]:
from pyspark.sql import functions as F

age_count = master_df.select('Age').distinct().count()
print(age_count)

In [ ]:
age_distribution = master_df.groupBy('Age').count().orderBy('Age')

age_distribution.show(age_distribution.count(), truncate=False)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit


# Define the age brackets
age_brackets = [(1, 19, 23), (2, 24, 28), (3, 29, 34), (4, 35, 39), (5, 40, 44)]

# Initialize the complete_df DataFrame
complete_df = master_df

# Create a new column 'age_bracket'
age_column = None  # Initialize age_column variable
for bracket in age_brackets:
    bracket_num, start_age, end_age = bracket
# Create a when condition for the current age bracket
    age_condition = (complete_df['Age'] >= start_age) & (complete_df['Age'] <= end_age)
# Update the age_column with the current bracket number
    age_column = when(age_condition, bracket_num).otherwise(age_column)

# Add the 'age_bracket' column to the DataFrame
complete_df = complete_df.withColumn('age_bracket', age_column)

# Show the updated DataFrame
complete_df.show()

In [ ]:
# # Group by the 'age_bracket' column, compute the count and sort the results by age_bracket
age_bracket_counts_sorted = complete_df.groupBy('age_bracket').count().orderBy('age_bracket')

# Show the count of players in each age bracket sorted by age_bracket
age_bracket_counts_sorted.show()

In [ ]:
# Define the output path including the file name
output_path = "MLB-player-archetypes-by-age.csv"

# Save the DataFrame as CSV
complete_df.write.csv(output_path, header=True, mode="overwrite")

In [ ]:
# Define the output path
output_path = "file:///C:/Users/18607/uu/homework/Project-4/baseballdatabank-2023.1.csv"

# Save the DataFrame as CSV
complete_df.write.csv(output_path, header=True, mode="overwrite")

In [1]:
import pandas as pd
import hvplot.pandas
from pathlib import Path

master_csv_path = Path("Resources/MLB-player-final-dataset.csv")

master_csv_read = pd.read_csv(master_csv_path)

master_summary_df = pd.DataFrame(master_csv_read)

master_summary_df.drop("Unnamed: 0", axis = 1, inplace = True)

master_summary_df['age_bracket'] = master_summary_df['age_bracket'].astype('category')

master_summary_df 

,IDfg,Season,Name,Age,G,AB,PA,BB%,K%,BB/K,...,Z-Swing%,O-Contact%,Z-Contact%,F-Strike%,SwStr%,Zone%,Clutch,UBR,WAR,age_bracket
0,18401,2023,Ronald Acuna Jr.,25,159,643,735,0.109,0.114,0.95,...,0.728,0.738,0.878,0.589,0.078,0.422,1.16,2.2,8.4,2
1,13611,2023,Mookie Betts,30,152,584,693,0.139,0.154,0.90,...,0.630,0.643,0.939,0.595,0.058,0.421,-0.72,2.3,8.3,3
2,5361,2023,Freddie Freeman,33,161,637,730,0.099,0.166,0.60,...,0.792,0.688,0.880,0.614,0.094,0.426,-1.50,1.4,7.9,3
3,14344,2023,Matt Olson,29,162,608,720,0.144,0.232,0.62,...,0.735,0.695,0.777,0.556,0.119,0.395,-0.60,2.3,6.7,3
4,19755,2023,Shohei Ohtani,28,135,497,599,0.152,0.239,0.64,...,0.707,0.565,0.814,0.571,0.139,0.393,-2.08,1.7,6.6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7562,2437,2013,Maicer Izturis,32,107,365,399,0.068,0.095,0.71,...,0.609,0.804,0.937,0.569,0.048,0.447,-0.31,0.5,-1.6,3
7563,3856,2013,Jeff Keppinger,33,117,423,451,0.044,0.091,0.49,...,0.569,0.819,0.970,0.572,0.034,0.482,-0.32,-3.6,-1.7,3
7564,8585,2013,Yuniesky Betancourt,31,137,391,409,0.034,0.174,0.20,...,0.750,0.704,0.878,0.670,0.100,0.462,0.84,-0.6,-1.8,3
7565,2677,2013,Jordan Pacheco,27,95,247,262,0.038,0.145,0.26,...,0.631,0.788,0.913,0.626,0.060,0.492,-0.19,-0.2,-1.8,2


In [2]:
scatter_plot_1 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "BB/K",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_1

:NdOverlay   [age_bracket]
   :Scatter   [Age]   (BB/K,Name)

In [ ]:
scatter_plot_2 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "maxEV",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_2

In [ ]:
scatter_plot_3 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "HardHit",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_3

In [ ]:
scatter_plot_4 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "Barrels",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_4

In [ ]:
scatter_plot_5 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "OBP",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_5

In [ ]:
scatter_plot_6 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "SLG",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_6

In [ ]:
scatter_plot_7 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "OPS",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_7

In [ ]:
scatter_plot_8 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "ISO",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_8

In [ ]:
scatter_plot_9 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "BABIP",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_9

In [ ]:
scatter_plot_10 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "O-Swing%",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_10

In [ ]:
scatter_plot_11 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "Z-Swing%",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_11

In [ ]:
scatter_plot_12 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "F-Strike%",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_12

In [ ]:
scatter_plot_13 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "Clutch",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_13

In [ ]:
scatter_plot_14 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "UBR",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_14

In [ ]:
scatter_plot_15 = master_summary_df.hvplot.scatter(
    x = "Age",
    y = "WAR",
    by = "age_bracket",
    hover_cols = ['Name']
)

scatter_plot_15

In [7]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Assuming 'Name' is the column with non-numeric data
master_summary_df['Name'] = master_summary_df['Name'].astype(str)

# Dropping the 'Name' column
master_summary_numeric_df = master_summary_df.drop(columns=['Name'])

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
master_summary_numeric_imputed = imputer.fit_transform(master_summary_numeric_df)

# Scale the data
scaler = StandardScaler()
master_summary_numeric_scaled = scaler.fit_transform(master_summary_numeric_imputed)

# Apply KMeans clustering
model = KMeans(n_clusters=5, random_state=0)
model.fit(master_summary_numeric_scaled)


/Users/jesseolivarez/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=5, random_state=0)

In [8]:
k_predict = model.fit(master_summary_numeric_scaled)
k_predict

/Users/jesseolivarez/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=5, random_state=0)

In [11]:
# Assuming 'Name' is the column with non-numeric data
master_summary_df['Name'] = master_summary_df['Name'].astype(str)

# Dropping the 'Name' column
master_summary_numeric_df = master_summary_df.drop(columns=['Name'])

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
master_summary_numeric_imputed = imputer.fit_transform(master_summary_numeric_df)

# Scale the data
scaler = StandardScaler()
master_summary_numeric_scaled = scaler.fit_transform(master_summary_numeric_imputed)

# Apply KMeans clustering
model = KMeans(n_clusters=5, random_state=0)
k_predict = model.fit_predict(master_summary_numeric_scaled)

# Create a new DataFrame for predictions
prediction_df = pd.DataFrame(data=master_summary_numeric_scaled, columns=master_summary_numeric_df.columns)

# Add the 'prediction' column
prediction_df['prediction'] = k_predict

# Show the resulting DataFrame
print(prediction_df.head())

/Users/jesseolivarez/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


       IDfg    Season       Age         G        AB        PA       BB%  \
0  1.184424  1.638039 -0.755116  1.759808  2.184747  2.261178  0.874802   
1  0.395063  1.638039  0.551829  1.618345  1.876359  2.064861  1.611864   
2 -0.964483  1.638039  1.335995  1.800226  2.153386  2.237807  0.629115   
3  0.515857  1.638039  0.290440  1.820436  2.001805  2.191065  1.734707   
4  1.407555  1.638039  0.029051  1.274792  1.421618  1.625486  1.931257   

         K%      BB/K       OBP  ...  O-Contact%  Z-Contact%  F-Strike%  \
0 -1.355893  2.494616  1.668836  ...    1.045617    0.480550  -0.479118   
1 -0.975676  2.289075  1.568765  ...    0.297801    1.289482  -0.397925   
2 -0.861610  1.055826  1.593783  ...    0.652030    0.507072  -0.140815   
3 -0.234251  1.138043  1.331096  ...    0.707132   -0.858829  -0.925678   
4 -0.167713  1.220259  1.618800  ...   -0.316195   -0.368166  -0.722696   

     SwStr%     Zone%    Clutch       UBR       WAR  age_bracket  prediction  
0 -0.866073 -0.4712

In [12]:
scatter_plot_16 = prediction_df.hvplot.scatter(
    x = "Age",
    y = "UBR",
    by = "prediction",
    hover_cols = ['Name']
)

scatter_plot_16

:NdOverlay   [prediction]
   :Scatter   [Age]   (UBR)

In [14]:
import plotly.express as px
prediction_df = prediction_df.reset_index()
figure = px.scatter_3d(
    prediction_df, 
    x = "Age",
    y = "UBR",
    z = "age_bracket",
    color = 'prediction',
    hover_data = ['index']
)

figure.show()